# Notebook for analysis of Lia's aims for undergrad dissertation 

In [9]:
library(psych)
library(ggplot2)
library(reshape2)
library(dplyr)


Attaching package: ‘dplyr’

The following object is masked from ‘package:reshape’:

    rename

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
data<-read.table("~/Google Drive/Lias_stuff/data/variables_interest.csv", sep=",", header=T, fill=T)

In [4]:
dim(data)

[1] 73 56

In [5]:
head(data)

PID,Assigned.Beverage,Sweet,w1_BMI,w1_BMI_status,age,ethnicity,race,gender,adlib_assigned,⋯,b5_tpbq_assigned_aa,b5_tpbq_assigned_ia,b5_tpbq_assigned_sn,b5_tpbq_assigned_pbc,b5_tpbq_assigned_bi,w2_tpbq_assigned_aa,w2_tpbq_assigned_ia,w2_tpbq_assigned_sn,w2_tpbq_assigned_pbc,w2_tpbq_assigned_bi
bbx_001,UCO,0,21.07869,0,22,0,2,0,0.1,⋯,NA,NA,NA,NA,NA,3.000000,5.000000,3.333333,4.666667,1.5
bbx_002,USL,0,34.55076,1,21,0,5,0,0.0,⋯,NA,NA,NA,NA,NA,4.333333,4.666667,4.666667,4.666667,7.0
bbx_003,USL,0,21.39481,0,19,0,5,0,0.0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
bbx_004,SL,1,20.37629,0,22,0,5,0,-0.2,⋯,2.666667,3.333333,3.333333,4.666667,5.5,3.000000,3.666667,3.333333,2.666667,5.0
bbx_005,CO,1,22.85640,0,20,0,5,0,2.3,⋯,3.000000,4.333333,4.666667,4.666667,1.5,3.000000,4.666667,5.000000,4.666667,7.0
bbx_006,USL,0,21.77146,0,21,0,5,0,97.6,⋯,3.000000,4.000000,3.333333,4.666667,5.0,3.333333,4.000000,3.333333,4.666667,5.5


## Code Dictionary
PID: bbx participant	
Assigned Beverage: Character type, UCO, USL, CO, SL

Sweet: assigned beverage sweetness, 1=SL/CO (sweet), 0=USL/UCO (unsweet)

w1_BMI: continuous, BMI based off W1 behavioral visit height and weight

w1_BMI_status: dichotomous, 0=below 25 BMI, 1=above 25 BMI

Age: continuous

Ethnicity: Hispanic/Latino, 0=no, 1=yes
Race: 
* 1      -american Indian or Alaska native
* 1-    Asian
* 2-    Black or African American
* 3-    Native Hawaiian or other specific islander
* 4-    White
* 5-    Middle eastern or northern African
* 6-    Other ____

Gender: 0=Male, 1=Female

### Naming breakdown:
* time data was taken at (w1= behavioral 1, b5= bevshop 5, w2= behavioral 2)
* TPB questionnaire
* Beverage (ssb=soda, usb=water, assigned= assigned beverage)
* TPB construct (aa= affective attitude, ia= instrumental attitude, sn= subjective norm, pbc= perceived behavioral control, bi= behavioral intention)
* i.e. w1_tpbq_ssb_aa = first behavioral, soda’s affective attitude score (mean of related questions)

## Variables of interest
#### Dependent variable
TPB constructs: 
* affective attitude (AA)
* instrumental attitude (IA)
* subjective norms (SN)
* perceived behavioral control (BC)
* behavioral intention (BI)
#### Independent variable
* Assigned Beverage Sweetness
#### Covariates
* BMI: continuous
* Age
* Gender
* Race
* Ethnicity
#### random effects
* Time
* PID



## Split dataset by the construct

In [10]:
names(data)

[1] "PID"                  "Assigned.Beverage"    "Sweet"               
 [4] "w1_BMI"               "w1_BMI_status"        "age"                 
 [7] "ethnicity"            "race"                 "gender"              
[10] "adlib_assigned"       "adlib_alt"            "w1_tpbq_ssb_aa"      
[13] "w1_tpbq_ssb_ia"       "w1_tpbq_ssb_sn"       "w1_tpbq_ssb_pbc"     
[16] "w1_tpbq_ssb_bi"       "b5_tpbq_ssb_aa"       "b5_tpbq_ssb_ia"      
[19] "b5_tpbq_ssb_sn"       "b5_tpbq_ssb_pbc"      "b5_tpbq_ssb_bi"      
[22] "w2_tpbq_ssb_aa"       "w2_tpbq_ssb_ia"       "w2_tpbq_ssb_sn"      
[25] "w2_tpbq_ssb_pbc"      "w2_tpbq_ssb_bi"       "w1_tpbq_usb_aa"      
[28] "w1_tpbq_usb_ia"       "w1_tpbq_usb_sn"       "w1_tpbq_usb_pbc"     
[31] "w1_tpbq_usb_bi"       "b5_tpbq_usb_aa"       "b5_tpbq_usb_ia"      
[34] "b5_tpbq_usb_sn"       "b5_tpbq_usb_pbc"      "b5_tpbq_usb_bi"      
[37] "w2_tpbq_usb_aa"       "w2_tpbq_usb_ia"       "w2_tpbq_usb_sn"      
[40] "w2_tpbq_usb_pbc"      "w2_tpbq_usb_bi"       "w1_tpbq_assigned_aa" 
[43] "w1_tpbq_assigned_ia"  "w1_tpbq_assigned_sn"  "w1_tpbq_assigned_pbc"
[46] "w1_tpbq_assigned_bi"  "b5_tpbq_assigned_aa"  "b5_tpbq_assigned_ia" 
[49] "b5_tpbq_assigned_sn"  "b5_tpbq_assigned_pbc" "b5_tpbq_assigned_bi" 
[52] "w2_tpbq_assigned_aa"  "w2_tpbq_assigned_ia"  "w2_tpbq_assigned_sn" 
[55] "w2_tpbq_assigned_pbc" "w2_tpbq_assigned_bi"

In [21]:
AA <- data %>% group_by(Sweet) %>% 
  select('PID','w1_BMI', 'w1_BMI_status', 
         'age', 'ethnicity', 'race', 
         'gender', 'w1_tpbq_ssb_aa', 
         'b5_tpbq_ssb_aa', 'w2_tpbq_ssb_aa', 
         'w1_tpbq_usb_aa', 'b5_tpbq_usb_aa',  'w2_tpbq_usb_aa')

Adding missing grouping variables: `Sweet`


In [22]:
dim(AA)
names(AA)

[1] 73 14

[1] "Sweet"          "PID"            "w1_BMI"         "w1_BMI_status" 
 [5] "age"            "ethnicity"      "race"           "gender"        
 [9] "w1_tpbq_ssb_aa" "b5_tpbq_ssb_aa" "w2_tpbq_ssb_aa" "w1_tpbq_usb_aa"
[13] "b5_tpbq_usb_aa" "w2_tpbq_usb_aa"

In [15]:
IA <- data %>% group_by(Sweet) %>% 
  select('PID','w1_BMI', 'w1_BMI_status', 
         'age', 'ethnicity', 'race', 
         'gender', 'w1_tpbq_ssb_ia', 
         'b5_tpbq_ssb_ia', 'w2_tpbq_ssb_aa', 
         'w1_tpbq_usb_aa', 'b5_tpbq_usb_aa',  'w2_tpbq_usb_aa')

ERROR: Error in parse(text = x, srcfile = src): <text>:2:19: unexpected string constant
1: AA <- data %>% group_by(Sweet) %>% 
2:   select('w1_BMI' 'w1_BMI_status'
                     ^
